In [8]:
import cv2 as cv
import numpy as np
from PIL import Image
from PIL import ImageTk
import tkinter as tk
import threading
import time

In [9]:
threshhold = 20
min_contour_area = 5000
frame_background = None #Biến chứa background tĩnh
frame_background_unblured = None
frame_current_global = None
frame_thresh_global = None
window = None
image_box = None
image_thresh = None
image_background = None
lock = threading.Lock()

In [10]:
class Frame_Diff_Processor(threading.Thread):
    def __init__(self, threadID, name, counter):
        threading.Thread.__init__(self)
        self.threadID = threadID
        self.name = name
        self.counter = counter   
    def run(self):
        camera = cv.VideoCapture(0)
        while True: 
            _, frame_current = camera.read() #Lấy frame từ video (webcam)      
            frame_gray = cv.cvtColor(frame_current, cv.COLOR_BGR2GRAY) #Chuyển sang ảnh xám    
            frame_gray = cv.GaussianBlur(frame_gray, (21, 21), 0)     #Đặt filter Gaussian để blur ảnh (xoá bớt nhiễu)   
            global frame_background, frame_background_unblured
            if frame_background is None: 
                frame_background_unblured = cv.cvtColor(frame_current, cv.COLOR_BGR2RGB) 
                frame_background = frame_gray 
                continue  
            frame_diff = cv.absdiff(frame_background, frame_gray) #Lấy difference
            global threshhold
            frame_thresh = cv.threshold(frame_diff, threshhold, 255, cv.THRESH_BINARY)[1] #Apply threshhold
            frame_thresh = cv.dilate(frame_thresh, None, iterations = 2) 
            frame_thresh = cv.medianBlur(frame_thresh,9)
            
            (_, contours, _) = cv.findContours(frame_thresh.copy(), cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE) 
            
            global min_contour_area
            for contour in contours: 
                if cv.contourArea(contour) > min_contour_area: 
                    (x, y, w, h) = cv.boundingRect(contour) 
                    cv.rectangle(frame_current, (x, y), (x + w, y + h), (0, 255, 0), 3) 

            lock.acquire()
            global frame_current_global, frame_thresh_global
            frame_current_global = frame_current
            frame_thresh_global = cv.resize(frame_thresh, (352,240))
            frame_current_global=cv.cvtColor(frame_current_global, cv.COLOR_BGR2RGB)           
            lock.release()      

        camera.release() 
        cv.destroyAllWindows() 

In [11]:
#Các hàm callback cho GUI
def set_threshhold(rate):
    global threshhold
    threshhold = int(rate)
def set_contour(area):
    global min_contour_area
    min_contour_area = int(area)
    
def recap_background():
    global frame_background
    frame_background = None
    
def change_image_box():
    global frame_current_global
    if frame_current_global is None:
        return
    global window, image_box   
    photo = ImageTk.PhotoImage(Image.fromarray(frame_current_global))
    image_box.configure(image=photo)
    image_box.image = photo
    image_box.grid(row=0, column=1, rowspan = 2, columnspan = 3)
    window.after(50, change_image_box)

def change_image_thresh():
    global frame_thresh_global
    if frame_thresh_global is None:
        return
    global window, image_thresh  
    photo = ImageTk.PhotoImage(Image.fromarray(frame_thresh_global))
    image_thresh.configure(image=photo)
    image_thresh.image = photo
    image_thresh.grid(row=1, column=0)
    window.after(50, change_image_thresh)
    
def change_image_background():
    global frame_background_unblured
    if frame_background_unblured is None:
        return
    global window, image_background  
    background_resize = cv.resize(frame_background_unblured, (352,240))
    photo = ImageTk.PhotoImage(Image.fromarray(background_resize))
    image_background .configure(image=photo)
    image_background .image = photo
    image_background .grid(row=0, column=0)
    window.after(50, change_image_background )

In [12]:
thread1 = Frame_Diff_Processor(1, "Thread-1", 1)
thread1.start()
time.sleep(2)

Exception in thread Thread-1:
Traceback (most recent call last):
  File "c:\users\nguyenhoangloc\appdata\local\programs\python\python36-32\lib\threading.py", line 916, in _bootstrap_inner
    self.run()
  File "<ipython-input-3-9ff8ecb197f7>", line 11, in run
    frame_gray = cv.cvtColor(frame_current, cv.COLOR_BGR2GRAY) #Chuyển sang ảnh xám
cv2.error: OpenCV(3.4.3) C:\projects\opencv-python\opencv\modules\imgproc\src\color.cpp:181: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'




In [13]:
window = tk.Tk()
window.title('Frame Differencing')

image_box = tk.Label(window)
image_thresh = tk.Label(window)
image_background = tk.Label(window)

tk.Button(window, 
           text="Recap Background", 
           command = recap_background).grid(row=3, column=0)

scale_thresh = tk.Scale(window, 
                        orient ='horizontal', 
                        from_ = 0, to = 255,
                        length = 200,
                        sliderlength = 60,
                        resolution = 5,
                        command = set_threshhold)
scale_thresh.grid(row=3, column=1)
scale_thresh.set(threshhold)

scale_contour = tk.Scale(window, 
                        orient ='horizontal', 
                        from_ = 0, to = 5000,
                        length = 200,
                        sliderlength = 60,
                        resolution = 50,
                        command = set_contour)
scale_contour.grid(row=3, column=2)
scale_contour.set(min_contour_area)

tk.Label(window,
        text = 'Adjust threshhold').grid(row=4, column=1)

tk.Label(window,
        text = 'Adjust min contour area').grid(row=4, column=2)

In [14]:
window.after(50, change_image_box)
window.after(50, change_image_thresh)
window.after(50, change_image_background)
window.mainloop()